In [21]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import cosine_distances
import gensim
from gensim.models import LsiModel
from gensim.matutils import kullback_leibler, jensen_shannon
from gensim.similarities import MatrixSimilarity
from scipy.sparse import csr_matrix
from gensim.models import TfidfModel
from gensim.models.coherencemodel import CoherenceModel
import nltk
from nltk.tokenize import word_tokenize

In [2]:
# Leitura dos dados do dataframe
#df = pd.read_csv('datasets/(processado-final)textos_tuitesPt_2020.csv.gz', names=['texto'])
df = pd.read_csv('datasets/(processado)textos_tuitesPt_2020_0.csv', names=['texto'])

# Elimina um valor flutuante que aparece no dataframe (por razões misteriosas)
# o algoritmo não aceita o valor flutuante, que precisa ser filtrado
df = df[df['texto'].apply(lambda x: isinstance(x, str))]
df['texto'].apply(type).value_counts()

df

,texto
0,coronavirus aceitar braco abrir
1,achar eleitor bolsonaro medo coronavirus febre...
2,fome coronavirus entrar fila
3,trancar replies twetr sala coronavirus
4,caso coronavirus confirmar Brasil mundo querer...
...,...
99996,preciso comando vermelho decretar toque recolh...
99997,urgente reporter confirmar segundo morte coron...
99998,informativo elaborar equipe viano azevedo advo...
99999,vivo Paulo confirmar primeiro morte covid19 Br...


In [3]:
tfidf_vectorizer = TfidfVectorizer(stop_words=None, 
                                    max_df=0.7, 
                                    min_df=5, 
                                    use_idf=True, 
                                    smooth_idf=True, 
                                    sublinear_tf=True)
matriz_tfidf = tfidf_vectorizer.fit_transform(df['texto'])

In [4]:
# Obtendo todos os documentos como uma lista
docs = df['texto'].tolist()

# Criando uma lista de palavras
words = []
for doc in docs:
    words += doc.split()

# Removendo palavras repetidas
terms = list(set(words))

In [5]:
corpus = gensim.matutils.Sparse2Corpus(matriz_tfidf.T)
dictionary = gensim.corpora.Dictionary.from_corpus(corpus, id2word=dict((i, term) for i, term in enumerate(terms)))

In [6]:
# Dividindo o corpus em conjuntos de treinamento e teste
num_docs = matriz_tfidf.shape[0]
train_docs = np.random.choice(num_docs, int(num_docs*0.8), replace=False)
test_docs = np.setdiff1d(range(num_docs), train_docs)

In [7]:
# Criando o modelo LSA para o conjunto de treinamento
num_topics = 10
lsa_model = gensim.models.LsiModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
train_corpus = gensim.matutils.Sparse2Corpus(matriz_tfidf[train_docs].T)
lsa_model = gensim.models.LsiModel(train_corpus, id2word=dictionary, num_topics=num_topics)

In [8]:
# Reconstruindo o corpus de teste a partir do modelo LSA
test_corpus = gensim.matutils.Sparse2Corpus(matriz_tfidf[test_docs].T)
reconstructed_corpus = []
for doc in test_corpus:
    reconstructed_doc = lsa_model[doc]
    reconstructed_corpus.append(reconstructed_doc)

In [9]:
for topic_num, topic_words in lsa_model.print_topics(num_topics=num_topics):
    print('Tópico {}:'.format(topic_num))
    words = topic_words.split()
    print(words)

Tópico 0:
['0.785*"abrigo"', '+', '0.223*"portr"', '+', '0.204*"iguaizinho"', '+', '0.136*"enviado"', '+', '0.126*"inhuman"', '+', '0.126*"bastioes"', '+', '0.088*"dgsaude"', '+', '0.088*"moratoria"', '+', '0.087*"daninha"', '+', '0.085*"caverno"']
Tópico 1:
['0.778*"portr"', '+', '-0.434*"abrigo"', '+', '0.282*"iguaizinho"', '+', '0.189*"bastioes"', '+', '0.134*"enviado"', '+', '0.095*"gabinetes"', '+', '-0.075*"inhuman"', '+', '0.074*"decoro"', '+', '0.066*"inhame"', '+', '0.061*"oficio"']
Tópico 2:
['0.619*"iguaizinho"', '+', '0.455*"bastioes"', '+', '-0.365*"portr"', '+', '0.261*"enviado"', '+', '-0.175*"gabinetes"', '+', '-0.166*"inhuman"', '+', '0.090*"inhame"', '+', '-0.089*"abominavel"', '+', '-0.086*"moratoria"', '+', '-0.082*"transfiram"']
Tópico 3:
['-0.680*"gabinetes"', '+', '-0.345*"abominavel"', '+', '0.318*"portr"', '+', '-0.306*"these"', '+', '-0.234*"inhuman"', '+', '0.218*"abrigo"', '+', '-0.137*"enviado"', '+', '-0.124*"iguaizinho"', '+', '-0.100*"etc"', '+', '-0.093

In [10]:
# Calculando a perplexidade
#perplexity = gensim.models.CoherenceModel(topics=[lsa_model.show_topic(i) for i in range(num_topics)], corpus=reconstructed_corpus, dictionary=dictionary, coherence='u_mass').get_coherence()

In [11]:
# Obtém a matriz de tópicos
topic_matrix = lsa_model.get_topics()
topic_matrix = topic_matrix.T

# Calcula a distância de cosseno entre todos os pares de tópicos
distance_matrix = cosine_distances(topic_matrix)

# Calcula a mediana da matriz de distância de tópicos
median_distance = np.median(distance_matrix)

# Imprime a mediana da matriz de distância de tópicos
print("A distância de tópicos: ", median_distance)

In [13]:
# Cria um objeto TfidfModel a partir do corpus
tfidf_model = TfidfModel(corpus)

# Calcula a matriz TF-IDF para o corpus
corpus_tfidf = tfidf_model[corpus]

# Calcula a divergência KL simétrica entre todos os pares de tópicos
similarity_matrix = MatrixSimilarity(lsa_model[corpus_tfidf], num_features=num_topics)
kl_divergences = similarity_matrix.get_similarities(topic_matrix)

# Substitui NaNs por zeros
#kl_divergences = np.nan_to_num(kl_divergences)

# Calcula a média da matriz de divergência KL simétrica
mean_kl_divergence = np.mean(kl_divergences)

# Imprime a média da matriz de divergência KL simétrica
print("Média da matriz de diverência KL simétrica: ", mean_kl_divergence)

In [18]:
# Cria um objeto CoherenceModel para calcular a coerência C_umass
coherence_model = CoherenceModel(model=lsa_model, corpus=corpus_tfidf, dictionary=dictionary, coherence='u_mass')

# Calcula a coerência C_umass
umass_coherence = coherence_model.get_coherence()

print("A coerência C_umass é: ", umass_coherence)

A coerência C_umass é:  -4.120386072467498


In [26]:
textos_preprocessados = []
for texto in docs:
    # Tokenização das palavras
    tokens = word_tokenize(texto)
    textos_preprocessados.append(tokens)

# Cria um objeto CoherenceModel para calcular a coerência C_w2v
coherence_model = CoherenceModel(model=lsa_model, texts=textos_preprocessados, dictionary=dictionary, coherence='c_w2v', window_size=10)

# Calcula a coerência C_w2v
w2v_coherence = coherence_model.get_coherence()

print("A coerência C_w2v é: ", w2v_coherence)

A coerência C_w2v é:  0.90548134
